Импорт необходимых библиотек. Проверьте, что всё стоит.
Если данные предварительно не прогнали через Geocoder, то необходимо это сделать.
На данный момент версия геокодера из pip (pip install sloyka) пригодна для работы только с второй таблицей, первая таблица через него будет построена неправильно.
Если у вас скачена последняя версия библиотеки с github() с ветки dev, то по идее достаточно просто положить блокнот в папку sloyka\src\


In [1]:
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm
from keybert import KeyBERT
import random
import re
import nltk
from nltk.corpus import stopwords
import numpy as np

from sloyka import Geocoder # или from geocoder import Geocoder, если не настроен импорт через sloyka ещё может быть вариант sloyka.src.geocoder

nltk.download('stopwords')

RUS_STOPWORDS = stopwords.words('russian') + ['фото', 'улица', 'дом', 'проспект',
                                              'дорога', 'час', 'год', 'утро',
                                              'здравствуйте', 'ул', 'пр', 'здание',
                                              'город', 'аноним', 'утро', 'день',
                                              'вечер']

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
I:\sloyka\venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Инициируем необходимые функции.

In [2]:
from pathlib import Path

directory = "C:\\Users\\thebe\\OneDrive - ITMO UNIVERSITY\\НИРМА\\Data\\Rawdata\\new_parser"
path_list = Path(directory).glob('*.csv')

df = pd.DataFrame()

for path in path_list:

    tmp_df = pd.read_csv(path, encoding='utf-8')
    df = pd.concat([df, tmp_df])

In [3]:
df = df.reset_index()

In [4]:
for i in range(len(df)):
    tmp = df['parents_stack'].iloc[i]
    if tmp is not np.nan and tmp is not int:
        if len(tmp) > 2:
            df.at[i, 'parents_stack'] = int(tmp[1:len(tmp)-1])
        else:
            df.at[i, 'parents_stack'] = np.nan

In [5]:
df

,index,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack
0,0,2024-03-27 09:00:01,232611,РАБОТА в г. СЕСТРОРЕЦКЕООО “Вириал” (машиностр...,260.0,0.0,0.0,post,https://clck.ru/34cGGQ#работавсестрорецке,NaN,NaN
1,1,2024-03-27 08:21:35,232609,С Днем театра всех! И кто знает - остались ли ...,360.0,0.0,0.0,post,NaN,NaN,NaN
2,2,2024-03-27 07:49:43,232606,В пригороде Петербурга начали токовать ТЕТЕРЕВ...,752.0,4.0,0.0,post,https://vk.com/video-147416952_456239386,NaN,NaN
3,3,2024-03-27 07:49:22,232605,"Утро туманное, утро седое",676.0,4.0,0.0,post,NaN,NaN,NaN
4,4,2024-03-26 22:42:51,232598,Чей котейка в Дубках бегает?,1332.0,15.0,2.0,post,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
253570,761,2022-12-01 05:34:05,169982,"[id138219748|Сергей], спасибо 🤗",NaN,0.0,NaN,reply,NaN,169967.0,169981
253571,762,2022-11-29 18:18:54,169957,"Пытались поджечь?\n\nИ хорошо что не горит, а ...",NaN,1.0,NaN,comment,NaN,169955.0,NaN
253572,763,2022-11-29 18:59:49,169961,NaN,NaN,0.0,NaN,reply,NaN,169955.0,169957
253573,764,2022-11-30 06:27:16,169964,Дорого,NaN,0.0,NaN,comment,NaN,169920.0,NaN


In [6]:
def clean_from_dublicates(data: pd.DataFrame or gpd.GeoDataFrame,
                              id_column: str
                              ) -> pd.DataFrame or gpd.GeoDataFrame:

    uniq_df = data.drop_duplicates(subset=[id_column], keep='first')
    uniq_df = uniq_df.reset_index(drop=True)

    return uniq_df

def clean_from_digits(data: pd.DataFrame or gpd.GeoDataFrame,
                          text_column: str
                          ) -> pd.DataFrame or gpd.GeoDataFrame:

    for i in range(len(data)):
        text = str(data[text_column].iloc[i]).lower()
        cleaned_text = ''.join([j for j in text if not j.isdigit()])

        data.at[i, text_column] = cleaned_text

    return data


def clean_from_toponims(data: pd.DataFrame or gpd.GeoDataFrame,
                            text_column: str,
                            name_column: str,
                            toponim_type_column: str
                            ) -> pd.DataFrame or gpd.GeoDataFrame:

    for i in range(len(data)):

        text = str(data[text_column].iloc[i]).lower()
        word_list = text.split()
        toponims = [str(data[name_column].iloc[i]).lower(), str(data[toponim_type_column].iloc[i]).lower()]

        text = ' '.join([j for j in word_list if j not in toponims])

        data.at[i, text_column] = text

    return data


def clean_from_links(data: pd.DataFrame or gpd.GeoDataFrame,
                         text_column: str
                         ) -> pd.DataFrame or gpd.GeoDataFrame:

    for i in range(len(data)):
        text = str(data[text_column].iloc[i])
        if '[' in text and ']' in text:
            start = text.index('[')
            stop = text.index(']')

            text = text[:start] + text[stop:]

        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

        data.at[i, text_column] = text

    return data

In [7]:
len(df)

253575

In [8]:
df = df.dropna(subset='text')
len(df)

252768

In [9]:
df.loc[df['text'] == None]

,index,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack


In [10]:
len(df)
df_1 = df[:len(df)//4]

#Таблица 1

Загружаем данные. Если есть df сразу после парсера, то загружать ничего не нужно.

In [11]:
file = "C:\\Users\\thebe\\Downloads\\Telegram Desktop\\ruina.geojson" # на винде путь через двойной \\
gdf = gpd.read_file(file)

In [12]:
df = clean_from_links(df, 'text')

C:\Users\thebe\AppData\Local\Temp\ipykernel_14156\2242205876.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.at[i, text_column] = text


In [13]:
len(df)

253574

In [14]:
df = df.replace('', None)
df.loc[df['text'] == '']
len(df)

253574

Или, если данные, сохранённые после парсера.

In [15]:
# df = pd.read_csv('путь', sep='; или другой разделитель', encoding='utf-8')
g = Geocoder(device='cuda')
gdf = g.run(df, 'text')

2024-03-28 22:17:07,722 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS
03-28 22:17 | INFO     | clear_names started
03-28 22:17 | INFO     | get_street started
03-28 22:17 | INFO     | extract_ner_street started


 99%|█████████▉| 252247/253574 [32:23<00:11, 110.75it/s]

2024-03-28 22:49:54,991 Warning: An empty Sentence was created! Are there empty strings in your dataset?


100%|█████████▉| 252313/253574 [32:23<00:11, 113.05it/s]

2024-03-28 22:49:55,548 Warning: An empty Sentence was created! Are there empty strings in your dataset?


...

03-28 23:10 | INFO     | extract_toponym started



100%|██████████| 253574/253574 [00:23<00:00, 10828.26it/s]

03-28 23:11 | INFO     | get_stem started


03-28 23:27 | INFO     | create_gdf started


100%|██████████| 270751/270751 [5:01:26<00:00, 14.97it/s]   


Смотрим, что всё прогналось и выявились топонимы.
На всякий случай сделаем копию и будем работать с ней, чтобы если что не ждать снова, пока Geocoder отработает.

In [16]:
gdf_back_up = gdf.copy()
gdf_back_up

,index,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,...,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0.0,2024-03-27 09:00:01,232611.0,РАБОТА в г. СЕСТРОРЕЦКЕООО “Вириал” (машиностр...,260.0,0.0,0.0,post,https://clck.ru/34cGGQ#работавсестрорецке,NaN,...,0.998,,Воскова,улица,улица Воскова Санкт-Петербург Россия,['улица Воскова Санкт-Петербург Россия'],улица Воскова Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767)
1,1.0,2024-03-27 08:21:35,232609.0,С Днем театра всех! И кто знает - остались ли ...,360.0,0.0,0.0,post,NaN,NaN,...,NaN,,Курортном районе,None,None,None,None,None,None,None
2,2.0,2024-03-27 07:49:43,232606.0,В пригороде Петербурга начали токовать ТЕТЕРЕВ...,752.0,4.0,0.0,post,https://vk.com/video-147416952_456239386,NaN,...,NaN,,Гатчинском парке,None,None,None,None,None,None,None
3,3.0,2024-03-27 07:49:22,232605.0,"Утро туманное, утро седое",676.0,4.0,0.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
4,4.0,2024-03-26 22:42:51,232598.0,Чей котейка в Дубках бегает?,1332.0,15.0,2.0,post,NaN,NaN,...,NaN,,Дубках,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270746,NaN,NaN,NaN,"], когда? На прошлой неделе еще был",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
270747,NaN,NaN,NaN,Спасибо за размещение,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
270748,NaN,NaN,NaN,"], на Смоленском православном запрещено, а на ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,Смоленском православном,None,None,None,None,None,None,None
270749,NaN,NaN,NaN,У мемориала Большое Заречье отдыхает стая ЛЕБЕ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,Большое Заречье,None,None,None,None,None,None,None


Очищаем текст

In [17]:
gdf = clean_from_dublicates(data=gdf,
                               id_column='id')
gdf = clean_from_digits(gdf,
                           'text')
gdf = clean_from_toponims(gdf,
                             'text',
                             'initial_street',
                             'Toponims')
gdf = clean_from_links(gdf,
                          text_column='text')

Инициализируем модель киберта

In [18]:
model = KeyBERT(model='DeepPavlov/rubert-base-cased') # Или другая модель, вообще, надо потестировать

No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with MEAN pooling.


Создаём новые столбцы и заполняем пустыми значениями

In [19]:
gdf[['words', 'is_true']] = None

Извлекаем слова и запсисываем в gpd

In [20]:
for i in tqdm(range(len(gdf))):
    text = gdf['text'].loc[i]
    extraction = model.extract_keywords(text, top_n=1, stop_words=RUS_STOPWORDS)
    if extraction:
        gdf.at[i, 'words'] = extraction[0][0]

  0%|          | 0/232678 [00:00<?, ?it/s]

Проверяем, что всё записалось

In [21]:
gdf

,index,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,words,is_true
0,0.0,2024-03-27 09:00:01,232611.0,работа в г. сестрорецкеооо “вириал” (машиностр...,260.0,0.0,0.0,post,https://clck.ru/34cGGQ#работавсестрорецке,NaN,...,Воскова,улица,улица Воскова Санкт-Петербург Россия,['улица Воскова Санкт-Петербург Россия'],улица Воскова Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767),cggq,None
1,1.0,2024-03-27 08:21:35,232609.0,с днем театра всех! и кто знает - остались ли ...,360.0,0.0,0.0,post,NaN,NaN,...,Курортном районе,None,None,None,None,None,None,None,театр,None
2,2.0,2024-03-27 07:49:43,232606.0,в пригороде петербурга начали токовать тетерев...,752.0,4.0,0.0,post,https://vk.com/video-147416952_456239386,NaN,...,Гатчинском парке,None,None,None,None,None,None,None,vk,None
3,3.0,2024-03-27 07:49:22,232605.0,"утро туманное, утро седое",676.0,4.0,0.0,post,NaN,NaN,...,None,None,,[''],,,None,None,седое,None
4,4.0,2024-03-26 22:42:51,232598.0,чей котейка в бегает?,1332.0,15.0,2.0,post,NaN,NaN,...,Дубках,None,None,None,None,None,None,None,бегает,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232673,761.0,2022-12-01 05:34:05,169982.0,"], спасибо 🤗",NaN,0.0,NaN,reply,NaN,169967.0,...,None,None,,[''],,,None,None,спасибо,None
232674,762.0,2022-11-29 18:18:54,169957.0,"пытались поджечь? и хорошо что не горит, а то ...",NaN,1.0,NaN,comment,NaN,169955.0,...,None,None,,[''],,,None,None,мчс,None
232675,764.0,2022-11-30 06:27:16,169964.0,дорого,NaN,0.0,NaN,comment,NaN,169920.0,...,None,None,,[''],,,None,None,дорого,None
232676,765.0,2022-11-30 06:27:42,169966.0,на вайлдберис,NaN,0.0,NaN,comment,NaN,169920.0,...,None,None,,[''],,,None,None,вайлдберис,None


Устанавливаем сколько процентов от текста будем анализировать

In [42]:
percent = 10 # целое число от 0 до 100

num_to_extract = len(gdf) * percent//100 # всего будет извлечено строк

Выделяем посты в новый gdf

In [23]:
posts = gdf.loc[gdf['type'] == 'post']

Очищаем от пустых значений, где слова не извлеклись

In [24]:
posts = posts.dropna(subset='words').reset_index()

Создаём список случайных индексов для заполнения для постов

In [25]:
post_indexes_to_check = []

while len(post_indexes_to_check) < num_to_extract//2:
  index = random.randint(0, len(posts))
  if index not in post_indexes_to_check:
    post_indexes_to_check.append(index)

Всё тоже самое для комментариев

In [26]:
comments = gdf.loc[gdf['type'] != 'post']

In [27]:
comments = comments.dropna(subset='words').reset_index()

In [28]:
comment_indexes_to_check = []

while len(comment_indexes_to_check) < num_to_extract//2:
  index = random.randint(0, len(posts))
  if index not in comment_indexes_to_check:
    comment_indexes_to_check.append(index)

Складываем, чтобы получить все нужные нам индексы

In [29]:
indexes_to_check = post_indexes_to_check + comment_indexes_to_check

Извлекаем все нужные значения по индексам из сходных данных

In [30]:
gdf_to_table = gdf.iloc[indexes_to_check].reset_index()

Смотрим что получилось

In [31]:
gdf_to_table

,level_0,index,date,id,text,views.count,likes.count,reposts.count,type,link,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,words,is_true
0,20497,8200.0,2022-11-11 10:07:50,269772.0,"поставят надувной каток, когда все успокоится ...",NaN,0.0,NaN,reply,NaN,...,None,None,,[''],,,None,None,успокоится,None
1,9143,377.0,2023-03-24 10:18:43,20517.0,"🌤 плюсовая температура, умеренный ветер, преим...",661.0,4.0,0.0,post,NaN,...,None,None,,[''],,,None,None,делаемпетербургчище,None
2,25033,144.0,2024-01-09 22:52:00,72037.0,"ретро-автомобили возле ""окея"" 🚗 видео от подпи...",4906.0,47.0,4.0,post,NaN,...,Богатырский,None,Богатырский проспект Санкт-Петербург Россия,['Богатырский проспект Санкт-Петербург Россия'],Богатырский проспект Санкт-Петербург Россия,Богатырский проспект,"Богатырский проспект, Северо-Приморская часть,...",POINT (30.23542 60.00397),окея,None
3,17731,5407.0,2023-06-30 19:11:52,304148.0,"], да, цепочка: гужа - жилищный комитет - фкр.",NaN,0.0,NaN,reply,NaN,...,None,None,,[''],,,None,None,фкр,None
4,9485,719.0,2024-03-16 17:52:09,28043.0,"], добрый день! список избирательных участков ...",NaN,2.0,NaN,comment,NaN,...,РФ,None,None,None,None,None,None,None,reg_kirovsk,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23261,15850,3521.0,2024-02-17 09:35:27,329428.0,именно так ☺️,NaN,0.0,NaN,comment,NaN,...,None,None,,[''],,,None,None,именно,None
23262,13630,1294.0,2024-03-18 17:07:06,333080.0,"], спасибо",NaN,7.0,NaN,comment,NaN,...,None,None,,[''],,,None,None,спасибо,None
23263,9321,555.0,2024-03-25 08:36:55,28308.0,"], дома нужно сидеть, посетителям в такое время!",NaN,2.0,NaN,comment,NaN,...,None,None,,[''],,,None,None,нужно,None
23264,1964,1969.0,2023-11-06 22:23:33,225449.0,ещё мельче сделать чертеж могли???,NaN,1.0,NaN,comment,NaN,...,None,None,,[''],,,None,None,могли,None


Формируем gdf только с нужными нам столбцами

In [32]:
result_df = gdf_to_table[['id', 'text', 'words', 'is_true']]

In [33]:
result_df

,id,text,words,is_true
0,269772.0,"поставят надувной каток, когда все успокоится ...",успокоится,None
1,20517.0,"🌤 плюсовая температура, умеренный ветер, преим...",делаемпетербургчище,None
2,72037.0,"ретро-автомобили возле ""окея"" 🚗 видео от подпи...",окея,None
3,304148.0,"], да, цепочка: гужа - жилищный комитет - фкр.",фкр,None
4,28043.0,"], добрый день! список избирательных участков ...",reg_kirovsk,None
...,...,...,...,...
23261,329428.0,именно так ☺️,именно,None
23262,333080.0,"], спасибо",спасибо,None
23263,28308.0,"], дома нужно сидеть, посетителям в такое время!",нужно,None
23264,225449.0,ещё мельче сделать чертеж могли???,могли,None


Сохраняем в удобный формат

In [35]:
result_df.to_csv('firs_table.csv', sep=';', encoding='utf-8')

#Таблица 2

Берём наш изначальный gdf

In [139]:
file = "C:\\Users\\thebe\\Downloads\\Telegram Desktop\\ruina.geojson"
gdf = gpd.read_file(file)

Или если работаем с теми же данными

In [103]:
gdf = gdf_back_up

Избавляемся от ненужных строк (где не геокодер нашёл топоним)

In [104]:
len(gdf)

270751

In [105]:
gdf = gdf.dropna(subset='only_full_street_name')

In [106]:
len(gdf)

242379

In [108]:
df_from_geo = pd.DataFrame(gdf)
df_from_geo = df_from_geo.reset_index()
df_from_geo

,level_0,index,date,id,text,views.count,likes.count,reposts.count,type,link,...,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0,0.0,2024-03-27 09:00:01,232611.0,РАБОТА в г. СЕСТРОРЕЦКЕООО “Вириал” (машиностр...,260.0,0.0,0.0,post,https://clck.ru/34cGGQ#работавсестрорецке,...,0.998,,Воскова,улица,улица Воскова Санкт-Петербург Россия,['улица Воскова Санкт-Петербург Россия'],улица Воскова Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767)
1,3,3.0,2024-03-27 07:49:22,232605.0,"Утро туманное, утро седое",676.0,4.0,0.0,post,NaN,...,NaN,,None,None,,[''],,,None,None
2,8,8.0,2024-03-26 12:34:40,232565.0,Местная администрация города Сестрорецка уведо...,2390.0,13.0,9.0,post,NaN,...,0.996,,Приморское,None,Приморское шоссе Санкт-Петербург Россия,['Приморское шоссе Санкт-Петербург Россия'],Приморское шоссе Санкт-Петербург Россия,Приморское шоссе,"Приморское шоссе, Лахта, округ Лахта-Ольгино, ...",POINT (30.17560 59.98921)
3,9,9.0,2024-03-26 12:08:51,232564.0,ВНИМАНИЕ! ПОМОГИТЕ НАЙТИ ПОДРОСТКА! Максимальн...,2640.0,22.0,9.0,post,NaN,...,NaN,,None,None,,[''],,,None,None
4,12,12.0,2024-03-25 13:57:46,232514.0,Фотофакт,2379.0,18.0,2.0,post,NaN,...,NaN,,None,None,,[''],,,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242374,270744,NaN,NaN,NaN,"], не переживайте,кормим его очень хорошо., у ...",NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
242375,270745,NaN,NaN,NaN,"У них дворников нет у этой УК, т. е нет дворни...",NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
242376,270746,NaN,NaN,NaN,"], когда? На прошлой неделе еще был",NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
242377,270747,NaN,NaN,NaN,Спасибо за размещение,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None


In [109]:
for i in tqdm(range(len(df_from_geo))):
    if gdf.only_full_street_name.iloc[i] == '':
        gdf.at[i, 'only_full_street_name'] = None


  0%|          | 0/242379 [00:00<?, ?it/s]

C:\Users\thebe\AppData\Local\Temp\ipykernel_14156\3820647194.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf.at[i, 'only_full_street_name'] = None


In [114]:
df_from_geo['only_full_street_name'] = df_from_geo['only_full_street_name'].replace('', None)
df_from_geo.only_full_street_name.iloc[1]

In [117]:
print(len(df_from_geo))
df_from_geo = df_from_geo.dropna(subset='only_full_street_name')
print(len(df_from_geo))

242379
35235


In [138]:
df_from_geo = df_from_geo.reset_index()

In [140]:
df_from_geo = df_from_geo.drop(columns='index')

In [118]:
gdf.only_full_street_name.iloc[1] == ''

False

In [102]:
len(gdf)

96301

In [141]:
percent = 10 # целое число от 0 до 100

num_to_extract = len(df_from_geo) * percent//100 # всего будет извлечено строк

In [142]:
num_to_extract

3523

In [143]:
posts = gdf.loc[gdf['type'] == 'post']

In [144]:
posts = posts.reset_index()

In [145]:
post_indexes_to_check = []

while len(post_indexes_to_check) < num_to_extract//2:
  index = random.randint(0, len(posts))
  if index not in post_indexes_to_check:
    post_indexes_to_check.append(index)

In [146]:
comments = gdf.loc[gdf['type'] != 'post']

In [147]:
comments = comments.reset_index()

In [148]:
comment_indexes_to_check = []

while len(comment_indexes_to_check) < num_to_extract//2:
  index = random.randint(0, len(posts))
  if index not in comment_indexes_to_check:
    comment_indexes_to_check.append(index)

In [149]:
indexes_to_check = post_indexes_to_check + comment_indexes_to_check

In [150]:
len(indexes_to_check)

3522

Выделяем нужные столбцы

In [136]:
df_from_geo = df_from_geo.drop(columns=['index', 'level_0'])

In [151]:
df_from_geo

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,...,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,2024-03-27 09:00:01,232611.0,РАБОТА в г. СЕСТРОРЕЦКЕООО “Вириал” (машиностр...,260.0,0.0,0.0,post,https://clck.ru/34cGGQ#работавсестрорецке,NaN,NaN,...,0.998,,Воскова,улица,улица Воскова Санкт-Петербург Россия,['улица Воскова Санкт-Петербург Россия'],улица Воскова Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767)
1,2024-03-26 12:34:40,232565.0,Местная администрация города Сестрорецка уведо...,2390.0,13.0,9.0,post,NaN,NaN,NaN,...,0.996,,Приморское,None,Приморское шоссе Санкт-Петербург Россия,['Приморское шоссе Санкт-Петербург Россия'],Приморское шоссе Санкт-Петербург Россия,Приморское шоссе,"Приморское шоссе, Лахта, округ Лахта-Ольгино, ...",POINT (30.17560 59.98921)
2,2024-03-23 22:46:36,232462.0,Здравствуйте. Есть корм для рыбок Петушков в в...,4543.0,6.0,1.0,post,NaN,NaN,NaN,...,NaN,,Воскова,None,улица Воскова Санкт-Петербург Россия,['улица Воскова Санкт-Петербург Россия'],улица Воскова Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767)
3,2024-03-22 13:14:07,232246.0,Белая туча заволокла Сестрорецк. Туча сползла ...,6671.0,36.0,5.0,post,NaN,NaN,NaN,...,0.972,5а,Воскова,None,улица Воскова 5а Санкт-Петербург Россия,['улица Воскова 5а Санкт-Петербург Россия'],улица Воскова 5а Санкт-Петербург Россия,улица Воскова,"улица Воскова, Петроградская сторона, Введенск...",POINT (30.30505 59.95767)
4,2024-03-22 11:12:20,232240.0,ПОИСК РОДСТВЕННИКОВ. Здравствуйте. Ищу потомко...,2925.0,7.0,1.0,post,NaN,NaN,NaN,...,0.998,322,Приморское,None,Приморское шоссе 322 Санкт-Петербург Россия,['Приморское шоссе 322 Санкт-Петербург Россия'],Приморское шоссе 322 Санкт-Петербург Россия,Приморское шоссе,"322, Приморское шоссе, Разлив, Сестрорецк, Сан...",POINT (29.97837 60.09920)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35230,NaN,NaN,То есть колдобины от ЖД моста до Энергетиков н...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.939,,Энергетиков,None,"проспект Энергетиков Санкт-Петербург Россия,м...",['проспект Энергетиков Санкт-Петербург Россия...,проспект Энергетиков Санкт-Петербург Россия,мост Энергетиков,"проспект Энергетиков, округ Полюстрово, Санкт-...",POINT (30.43714 59.97155)
35231,NaN,NaN,То есть колдобины от ЖД моста до Энергетиков н...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.939,,Энергетиков,None,"проспект Энергетиков Санкт-Петербург Россия,м...",['проспект Энергетиков Санкт-Петербург Россия...,мост Энергетиков Санкт-Петербург Россия,проспект Энергетиков,"мост Энергетиков, Яблоновка, округ Малая Охта,...",POINT (30.43541 59.93633)
35232,NaN,NaN,То есть колдобины от ЖД моста до Энергетиков н...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.939,,Энергетиков,None,"проспект Энергетиков Санкт-Петербург Россия,м...",['проспект Энергетиков Санкт-Петербург Россия...,мост Энергетиков Санкт-Петербург Россия,мост Энергетиков,"мост Энергетиков, Яблоновка, округ Малая Охта,...",POINT (30.43541 59.93633)
35233,NaN,NaN,"], у этого вопроса есть предыстория:\n[id42808...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,Новостроек,None,улица Новостроек Санкт-Петербург Россия,['улица Новостроек Санкт-Петербург Россия'],улица Новостроек Санкт-Петербург Россия,улица Новостроек,"улица Новостроек, Автово, округ Автово, Санкт-...",POINT (30.26071 59.87681)


In [163]:
gdf = gdf_back_up.copy()

In [167]:
gdf = gdf.dropna(subset='only_full_street_name')
gdf_empty = gdf.loc[gdf['only_full_street_name'] == '']
gdf_empty

,index,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,...,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
3,3.0,2024-03-27 07:49:22,232605.0,"Утро туманное, утро седое",676.0,4.0,0.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
9,9.0,2024-03-26 12:08:51,232564.0,ВНИМАНИЕ! ПОМОГИТЕ НАЙТИ ПОДРОСТКА! Максимальн...,2640.0,22.0,9.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
12,12.0,2024-03-25 13:57:46,232514.0,Фотофакт,2379.0,18.0,2.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
14,14.0,2024-03-24 23:57:45,232501.0,Срочно!Убежала собака в районе улицы первого м...,2580.0,15.0,1.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
15,15.0,2024-03-24 19:00:00,232492.0,Вызов нарколог-Сестрорецк.рф на дом! 8(921)900...,1941.0,3.0,0.0,post,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270744,NaN,NaN,NaN,"], не переживайте,кормим его очень хорошо., у ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
270745,NaN,NaN,NaN,"У них дворников нет у этой УК, т. е нет дворни...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
270746,NaN,NaN,NaN,"], когда? На прошлой неделе еще был",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
270747,NaN,NaN,NaN,Спасибо за размещение,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None


In [170]:
empty_indexes = []

while len(empty_indexes) < 2000:
  index = random.randint(0, len(gdf_empty))
  if index not in empty_indexes:
    empty_indexes.append(index)

In [171]:
len(empty_indexes)

2000

In [173]:
empty_result_df = gdf_empty.iloc[empty_indexes].reset_index()

In [175]:
empty_result_df = empty_result_df[['id', 'text', 'only_full_street_name']]

In [177]:
columns = ['TP', 'TN', 'FP', 'FN']

for i in columns:
    empty_result_df[i] = None

C:\Users\thebe\AppData\Local\Temp\ipykernel_14156\1515476269.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_result_df[i] = None
C:\Users\thebe\AppData\Local\Temp\ipykernel_14156\1515476269.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_result_df[i] = None
C:\Users\thebe\AppData\Local\Temp\ipykernel_14156\1515476269.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [178]:
empty_result_df

,id,text,only_full_street_name,TP,TN,FP,FN
0,77699.0,"], приходите за сыром☺ ( он типо Адыгейского) ...",,None,None,None,None
1,4361369.0,"И у аптеки ЛекОптТорг на углу тоже скользко, е...",,None,None,None,None
2,263201.0,"], Вы пишете страшные вещи.\nЕсли для нас - по...",,None,None,None,None
3,149422.0,ВНИМАНИЕ! ПОМОГИТЕ НАЙТИ ЧЕЛОВЕКА! Максимальны...,,None,None,None,None
4,1604911.0,Бородаааааа,,None,None,None,None
...,...,...,...,...,...,...,...
1995,74742.0,Ребята понимают кто является гарантом их благо...,,None,None,None,None
1996,1417044.0,"Да, начальник один.",,None,None,None,None
1997,19697.0,"И вновь благодарим всех, кто принял участие в ...",,None,None,None,None
1998,1594619.0,"С дач вернулись пенсионеры скучно сидеть дома,...",,None,None,None,None


In [179]:
joined_result_df = pd.concat([result_df, empty_result_df])

In [180]:
joined_result_df

,id,text,only_full_street_name,TP,TN,FP,FN
0,78805.0,"У нас в Пушкине тоже сегодня игра,к нам ребята...",улица Пушкина,None,None,None,None
1,64866.0,"Здравствуйте!Дорогие соседи,рядом с ТК ""Парнас...",улица Меркурьева,None,None,None,None
2,17764.0,Праздник в кругу семьи! 🎉❄Семья - это крепость...,Невский проспект,None,None,None,None
3,4105977.0,Внутреннее кольцо КАД закрывают.Движение от Пр...,Приморский проспект,None,None,None,None
4,1554859.0,"знак весит, на многих весит ограничитель, но п...",Витебский мост,None,None,None,None
...,...,...,...,...,...,...,...
1995,74742.0,Ребята понимают кто является гарантом их благо...,,None,None,None,None
1996,1417044.0,"Да, начальник один.",,None,None,None,None
1997,19697.0,"И вновь благодарим всех, кто принял участие в ...",,None,None,None,None
1998,1594619.0,"С дач вернулись пенсионеры скучно сидеть дома,...",,None,None,None,None


In [152]:
result_df = df_from_geo.iloc[indexes_to_check].reset_index()

In [153]:
result_df = result_df[['id', 'text', 'only_full_street_name']]

Добавляем новые колонки и заполняем None

In [154]:
columns = ['TP', 'TN', 'FP', 'FN']

In [155]:
for i in columns:
    result_df[i] = None

Проверяем, что получилось

In [156]:
result_df

,id,text,only_full_street_name,TP,TN,FP,FN
0,78805.0,"У нас в Пушкине тоже сегодня игра,к нам ребята...",улица Пушкина,None,None,None,None
1,64866.0,"Здравствуйте!Дорогие соседи,рядом с ТК ""Парнас...",улица Меркурьева,None,None,None,None
2,17764.0,Праздник в кругу семьи! 🎉❄Семья - это крепость...,Невский проспект,None,None,None,None
3,4105977.0,Внутреннее кольцо КАД закрывают.Движение от Пр...,Приморский проспект,None,None,None,None
4,1554859.0,"знак весит, на многих весит ограничитель, но п...",Витебский мост,None,None,None,None
...,...,...,...,...,...,...,...
3517,928953.0,"], в Выборгском и я получаю розовую квитанцию…...",Выборгский переулок,None,None,None,None
3518,1672364.0,"], невозможно сожрать ухо без допинга, накачан...",улица Пушкина,None,None,None,None
3519,19135.0,я отправила заявку на указанную почту . Воп...,Песочное шоссе,None,None,None,None
3520,20233.0,🔔 В марте 2024 года пройдёт 17-й конкурс профе...,улица Ломоносова,None,None,None,None


Сохранем в файл

In [181]:
joined_result_df.to_csv('second_table.csv', sep=';', encoding='utf-8')